# Extract Answer

Read each student repo and save their source code and number of commit to excel.

In [ ]:
%pip install GitPython pandas openpyxl

In [ ]:
from datetime import datetime
base_folder = "data/ITE3101_practical_tests/ite-3101-practical-test-ab-submissions/"
code_files = ["lab/lab01/ch01_t01_q1.py", "lab/lab01/ch01_t02_q2.py",
              "lab/lab01/ch01_t03_q3.py", "lab/lab01/ch01_t04_q4.py"]
deadline = datetime.strptime("2023-11-27 16:00:00", "%Y-%m-%d %H:%M:%S")

In [ ]:
import os
from git import Repo

commit_times = {}

for directory in os.listdir(base_folder):
    directory_path = os.path.join(base_folder, directory)
    if os.path.isdir(directory_path):
        repo = Repo(directory_path)
        last_commit = repo.head.commit
        commit_times[directory] = last_commit.committed_datetime

commit_times

In [ ]:
import pandas as pd
from pytz import utc

df_commit_times = pd.DataFrame(commit_times.items(), columns=[
                               'Directory', 'Commit Time'])
df_commit_times['Commit Time'] = pd.to_datetime(
    df_commit_times['Commit Time'], utc=True)
# Convert deadline to datetime64[ns, UTC]
deadline = pd.to_datetime(deadline, utc=True)
df_commit_times['After Deadline'] = df_commit_times['Commit Time'] > deadline

df_commit_times

Check any student commit code after the deadline.

In [ ]:
filtered_df = df_commit_times[df_commit_times['After Deadline']]
filtered_df

In [ ]:
import os
from git import Repo


def get_number_of_commits(repo_path, file_path):
    repo = Repo(repo_path)
    commits = list(repo.iter_commits(paths=file_path))
    return len(commits)


for code_file in code_files:
    column_name = os.path.basename(code_file)
    file_content = []
    number_of_code_commit = []

    for directory in df_commit_times['Directory']:
        file_path = os.path.join(base_folder, directory, code_file)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                content = file.read()
                file_content.append(content)
                # get the number of GitHub commit for file_path
                number_of_code_commit.append(get_number_of_commits(
                    os.path.join(base_folder, directory), code_file))
        else:
            file_content.append(None)
            number_of_code_commit.append(0)
    df_commit_times[column_name + ' Content'] = file_content
    df_commit_times[column_name + ' Commit'] = number_of_code_commit


df_commit_times

In [ ]:
import os

# Convert timezone-aware datetime to timezone-naive
df_commit_times['Commit Time'] = df_commit_times['Commit Time'].dt.tz_convert(
    None)

excel_file_path = os.path.join(base_folder, 'student_answer.xlsx')
df_commit_times.to_excel(excel_file_path, index=False)